In [2]:
import json
with open('data-example/tmp.json') as f:
    raw_data = json.load(f)

In [3]:
X = raw_data[1]
Y = [raw_data[0][i][4:8] for i in range(len(X))]

In [4]:
X = [X, X, X]  # [3 * 448 * 65]
Y = [Y, Y, Y]  # [3 * 448 * 4]

In [5]:
music_len = 448
print len(X), len(X[0]), len(X[0][0])
print len(Y), len(Y[0]), len(Y[0][0])

3 448 65
3 448 4


In [6]:
import tensorflow as tf
data = tf.placeholder(tf.float32, [None, music_len, 65])
target = tf.placeholder(tf.float32, [None, music_len, 4])
num_hidden = 200
cell = tf.nn.rnn_cell.LSTMCell(num_hidden, state_is_tuple=True)
val, state = tf.nn.bidirectional_dynamic_rnn(cell, cell, data, sequence_length=[music_len], dtype=tf.float32)

In [7]:
weight = tf.Variable(tf.truncated_normal([num_hidden * 2, 4]))
bias = tf.Variable(tf.constant(0.0, shape=[4]))

In [8]:
val = tf.concat(2, val)
val = tf.transpose(val, [1, 0, 2])
target_tr = tf.transpose(target, [1, 0, 2])
preds = [tf.nn.softmax(tf.matmul(val[i], weight) + bias) for i in range(music_len)]

In [9]:
cross_ens = [
    tf.nn.softmax_cross_entropy_with_logits(preds[i], target_tr[i])
    for i in range(music_len)
]

In [10]:
cross_sum = tf.reduce_sum(cross_ens)

In [11]:
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_sum)

In [12]:
mistakes = [tf.not_equal(tf.argmax(target_tr[i], 1), tf.argmax(preds[i], 1)) 
            for i in range(music_len)]
errors = [tf.reduce_mean(tf.cast(mistakes[i], tf.float32)) for i in range(music_len)]
error = tf.reduce_mean(errors)

In [13]:
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

In [ ]:
batch_size = 1
no_of_batches = int(len(X) / batch_size)
epoch = 100
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = X[ptr:ptr+batch_size], Y[ptr:ptr+batch_size]
        ptr += batch_size
        sess.run(minimize, {data: inp, target: out})
    incorrect = sess.run(error, {data: X, target: Y})
    print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))
sess.close()